***GENERATED CODE FOR rideshare4 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ride_Date', 'transformation_label': 'Extract Date'}], 'feature': 'Ride_Date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
                                                'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Ride_Date'}, {'feature_label': 'Ride_Date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('Ride_Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'rideable_type', 'transformation_label': 'String Indexer'}], 'feature': 'rideable_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'docked_bike', 'max': 'docked_bike', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'rideable_type'}, {'feature_label': 'rideable_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('rideable_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'start_station_name', 'transformation_label': 'String Indexer'}], 'feature': 'start_station_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10th & E St NW', 'max': 'Key Blvd & N Quinn St', 'missing': '0', 'distinct': '302'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'start_station_name'}, {'feature_label': 'start_station_name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('start_station_name')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'start_station_id', 'threshold': 31386.716, 'transformation_label': 'Binarizer'}], 'feature': 'start_station_id', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '31386.72', 'stddev': '307.31', 'min': '31000.0', 'max': '32604.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'start_station_id'}, {'feature_label': 'start_station_id', 'threshold': 31386.716, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('start_station_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'member_casual', 'transformation_label': 'String Indexer'}], 'feature': 'member_casual', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'casual', 'max': 'member', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'member_casual'}, {'feature_label': 'member_casual', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('member_casual')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'preciptype', 'transformation_label': 'String Indexer'}], 'feature': 'preciptype', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'rain', 'max': 'rain', 'missing': '0', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'preciptype'}, {'feature_label': 'preciptype', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('preciptype')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'severerisk', 'transformation_label': 'String Indexer'}], 'feature': 'severerisk', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'severerisk'}, {'feature_label': 'severerisk', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('severerisk')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'sunrise', 'transformation_label': 'Extract Date'}], 'feature': 'sunrise', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'sunrise'}, {'feature_label': 'sunrise', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('sunrise')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'sunset', 'transformation_label': 'Extract Date'}], 'feature': 'sunset', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'sunset'}, {'feature_label': 'sunset', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('sunset')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'conditions', 'transformation_label': 'String Indexer'}], 'feature': 'conditions', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Rain, Partially cloudy', 'max': 'Rain, Partially cloudy', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'conditions'}, {'feature_label': 'conditions', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('conditions')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'description', 'transformation_label': 'String Indexer'}], 'feature': 'description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Partly cloudy throughout the day with rain clearing later.', 'max': 'Partly cloudy throughout the day with rain clearing later.', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'description'}, {'feature_label': 'description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('description')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'icon', 'transformation_label': 'String Indexer'}], 'feature': 'icon', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'cloudy', 'max': 'cloudy', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'icon'}, {'feature_label': 'icon', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('icon')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run rideshare4Hooks.ipynb
try:
	#sourcePreExecutionHook()

	rideshare = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/rideshare4.csv', 'filename': 'rideshare4.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run rideshare4Hooks.ipynb
try:
	#transformationPreExecutionHook()

	rideshareautofe = TransformationMain.run(rideshare,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Ride_Date", "transformation_label": "Extract Date"}], "feature": "Ride_Date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Ride_Date"}, {"transformationsData": [{"feature_label": "rideable_type", "transformation_label": "String Indexer"}], "feature": "rideable_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "docked_bike", "max": "docked_bike", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "rideable_type"}, {"transformationsData": [{"feature_label": "start_station_name", "transformation_label": "String Indexer"}], "feature": "start_station_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10th & E St NW", "max": "Key Blvd & N Quinn St", "missing": "0", "distinct": "302"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "start_station_name"}, {"transformationsData": [{"feature_label": "start_station_id", "threshold": 31386.716, "transformation_label": "Binarizer"}], "feature": "start_station_id", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "31386.72", "stddev": "307.31", "min": "31000.0", "max": "32604.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "start_station_id"}, {"transformationsData": [{"feature_label": "member_casual", "transformation_label": "String Indexer"}], "feature": "member_casual", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "casual", "max": "member", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "member_casual"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ride", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.37", "stddev": "4.21", "min": "1", "max": "28", "missing": "0"}, "updatedLabel": "ride"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.0", "stddev": "0.0", "min": "5", "max": "5", "missing": "0"}, "updatedLabel": "Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WeekNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "21.0", "stddev": "0.0", "min": "21", "max": "21", "missing": "0"}, "updatedLabel": "WeekNumber"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "Day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tempmax", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "19.9", "stddev": "0.0", "min": "19.9", "max": "19.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "tempmax"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tempmin", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.6", "stddev": "0.0", "min": "11.6", "max": "11.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "tempmin"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "temp", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "14.9", "stddev": "0.0", "min": "14.9", "max": "14.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "temp"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "feelslikemax", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "19.9", "stddev": "0.0", "min": "19.9", "max": "19.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "feelslikemax"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "feelslikemin", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "13.8", "stddev": "0.0", "min": "13.8", "max": "13.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "feelslikemin"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "feelslike", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "16.6", "stddev": "0.0", "min": "16.6", "max": "16.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "feelslike"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dew", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "12.4", "stddev": "0.0", "min": "12.4", "max": "12.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "dew"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "humidity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "77.4", "stddev": "0.0", "min": "77.4", "max": "77.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "humidity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "precip", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.61", "stddev": "0.0", "min": "2.607", "max": "2.607", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "precip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "precipprob", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "precipprob"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "precipcover", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "29.17", "stddev": "0.0", "min": "29.17", "max": "29.17", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "precipcover"}, {"transformationsData": [{"feature_label": "preciptype", "transformation_label": "String Indexer"}], "feature": "preciptype", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "rain", "max": "rain", "missing": "0", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "preciptype"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "snow", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "snow"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "snowdepth", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "snowdepth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "windgust", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "38.9", "stddev": "0.0", "min": "38.9", "max": "38.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "windgust"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "windspeed", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "27.8", "stddev": "0.0", "min": "27.8", "max": "27.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "windspeed"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "winddir", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "321.2", "stddev": "0.0", "min": "321.2", "max": "321.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "winddir"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sealevelpressure", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1017.3", "stddev": "0.0", "min": "1017.3", "max": "1017.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sealevelpressure"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cloudcover", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "96.6", "stddev": "0.0", "min": "96.6", "max": "96.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "cloudcover"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "visibility", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "16.0", "stddev": "0.0", "min": "16.0", "max": "16.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "visibility"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "solarradiation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "34.4", "stddev": "0.0", "min": "34.4", "max": "34.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "solarradiation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "solarenergy", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.8", "stddev": "0.0", "min": "2.8", "max": "2.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "solarenergy"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "uvindex", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "uvindex"}, {"transformationsData": [{"feature_label": "severerisk", "transformation_label": "String Indexer"}], "feature": "severerisk", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "severerisk"}, {"transformationsData": [{"feature_label": "sunrise", "transformation_label": "Extract Date"}], "feature": "sunrise", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "sunrise"}, {"transformationsData": [{"feature_label": "sunset", "transformation_label": "Extract Date"}], "feature": "sunset", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "sunset"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "moonphase", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.3", "stddev": "0.0", "min": "0.3", "max": "0.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "moonphase"}, {"transformationsData": [{"feature_label": "conditions", "transformation_label": "String Indexer"}], "feature": "conditions", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Rain, Partially cloudy", "max": "Rain, Partially cloudy", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "conditions"}, {"transformationsData": [{"feature_label": "description", "transformation_label": "String Indexer"}], "feature": "description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Partly cloudy throughout the day with rain clearing later.", "max": "Partly cloudy throughout the day with rain clearing later.", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "description"}, {"transformationsData": [{"feature_label": "icon", "transformation_label": "String Indexer"}], "feature": "icon", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "cloudy", "max": "cloudy", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "icon"}]}))

	#transformationPostExecutionHook(rideshareautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run rideshare4Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(rideshareautofe, ["Year", "Month", "WeekNumber", "Day", "tempmax", "tempmin", "temp", "feelslikemax", "feelslikemin", "feelslike", "dew", "humidity", "precip", "precipprob", "precipcover", "snow", "snowdepth", "windgust", "windspeed", "winddir", "sealevelpressure", "cloudcover", "visibility", "solarradiation", "solarenergy", "uvindex", "moonphase", "Ride_Date_dayofmonth", "Ride_Date_month", "Ride_Date_year", "rideable_type_stringindexer", "start_station_name_stringindexer", "start_station_id_binarizer", "member_casual_stringindexer", "preciptype_stringindexer", "severerisk_stringindexer", "sunrise_dayofmonth", "sunrise_month", "sunrise_year", "sunset_dayofmonth", "sunset_month", "sunset_year", "conditions_stringindexer", "description_stringindexer", "icon_stringindexer"], "ride")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

